In [10]:
!pip install bs4

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import requests as req
from bs4 import BeautifulSoup
import re
import string
from tqdm import tqdm_notebook

In [132]:
punctuation = '.,!?:;-'

def preprocess_text(text):
    text = text.lstrip()
    text = re.sub('\xa0', ' ', text)
    text = re.sub(r"\d+", '', text)
    text = re.sub(r'\([^()]*\)', '', text)
    text = text.replace('[]', '')
    text = re.sub(r'[«»]', '', text)
    #убрать все иностранные слова
    return text

In [10]:
def get_n_pages(link):
    r = req.get(link)
    soup = BeautifulSoup(r.content.decode("utf-8"), 'lxml')
    pages = soup.find_all('a', 'p')
    if len(pages):
        n_pages = int(pages[-1]['page'])
    else:
        n_pages = 1
    return n_pages

In [11]:
def get_book(link_for_book):
    url_init = 'https://www.litmir.me'
    url_for_books = url_init + link_for_book + '&p={}'
    book = ''
    n_pages = get_n_pages(url_for_books.format(1))
    
    for page in range(1, n_pages+1):
        url = url_for_books.format(page)
        r = req.get(url)
        soup = BeautifulSoup(r.content.decode("utf-8"), 'lxml')
        raw_text = soup.find('div','page_text')
        section_1 = raw_text.find('a', attrs = {'name':'section_1'})
        if section_1:
            section_1 = section_1.text
            text = raw_text.text.replace(section_1, '')
        else:
            text = raw_text.text
            
        text = preprocess_text(text)
        book += text
    
    return book

In [12]:
def get_dataset(links):
    dataset = ''
    for link in links:
        dataset += get_book(link)
    return dataset

In [33]:
def get_full_dataset(pages):
    url_init = 'https://www.litmir.me/bs/?g=sg56' + '&p={}'
    n_pages = min(pages, get_n_pages(url_init.format(1)))
    full_dataset = ''
    
    for page in tqdm_notebook(range(1, n_pages)):
        r = req.get(url_init.format(page))
        soup = BeautifulSoup(r.content.decode("utf-8"), 'lxml')
        all_links = soup.find_all('div', attrs = {'style': 'width:180px; padding-top: 10px; margin-bottom: 13px;'})
        links = []
        for link in all_links:
            try:
                links.append(link.find('a')['href'])
            except:
                pass
        full_dataset += get_dataset(links)
        
    return full_dataset

In [34]:
dataset = get_full_dataset(20)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
setup = set(re.findall(r'[^ЙйЦцУуКкЕеНнГгШшЩщЗзХхъФфЫыВвАаПпРрОоЛлДдЖжэЭЯяЧчСсМмИиТтьБбЮюёЁ]', dataset))-set('!,.-?:; ')
dataset = re.sub(r'[řЄU{iM“&È®΄ΣF′™‚−%χůεΟﬀαg̀γτúć‘‹Hѕł—žCÑvЈwκ<ěą\u2003½V_oЪëbΘLP|πŽ°ΙÌñEt♣ї”Ó@©·ηíβΕі~ČΥÚσ`×d…№KΛѣ¦ò£ΜBЇQʼҐlς\nDνο\\―„§c"ρ€•±φx(sW›–ﬂμºTÁp¾Š’ΡΝjÄιX>kЬZђRSG\xadefδΑśY)nІÍєÀўa\'*hč=zNλmqšā#ArЅﬁIuJ+¼¿^ґ}¡æ/јyωO]', '', dataset)
dataset = dataset.replace('[', '')
dataset = dataset.replace(']', '')

In [130]:
with open('dataset.txt', 'w', encoding='utf-8') as f:
    f.write(dataset)